In [9]:
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [10]:
# import the pickle file that contains training, validating, and testing dataset
pickle_file = 'breast.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (17280, 40, 40), (17280,))
('Validation set', (2880, 40, 40), (2880,))
('Test set', (2880, 40, 40), (2880,))


In [11]:
# reformat the data to have the correct dimension
height = 40
width = 40
num_labels = 2
num_channels = 1 # grayscale

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, width, height, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (17280, 40, 40, 1), (17280, 2))
('Validation set', (2880, 40, 40, 1), (2880, 2))
('Test set', (2880, 40, 40, 1), (2880, 2))


In [12]:
# calculate the accuracy of prediction as compared to the true labels
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [13]:
# generate the confusion matrix, and calculate the prediction probability
# returns a list of tuples, each tuple has the format: (true label, model prediction, prediction probability)
# labels = 0 means benign, corresponds to (1, 0) in the output
# labels = 1 means malignant, corresponds to (0, 1) in the output
# threshold for classification is 0.5, meaning if the predicted value is bigger than 0.5, 
#     it's classified as malignant; if the predicted value is smaller than 0.5, it's classified as benign. 
def confusion(predictions, labels):   
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    prob_list = []
    prob = 0
    for i in range(len(labels)):
        if labels[i][1] > labels[i][0] and predictions[i][1] > predictions[i][0]: 
            TP += 1
            prob = (1, 1, predictions[i][1] / (predictions[i][1] + predictions[i][0]))
        elif labels[i][1] < labels[i][0] and predictions[i][1] < predictions[i][0]: 
            TN += 1
            prob = (0, 0, predictions[i][0] / (predictions[i][1] + predictions[i][0]))
        elif labels[i][1] < labels[i][0] and predictions[i][1] > predictions[i][0]: 
            FP += 1
            prob = (0, 1, predictions[i][1] / (predictions[i][1] + predictions[i][0]))
        elif labels[i][1] > labels[i][0] and predictions[i][1] < predictions[i][0]: 
            FN += 1
            prob = (1, 0, predictions[i][0] / (predictions[i][1] + predictions[i][0]))
        prob_list.append(prob)
    print('TP, TN, FP, FN: (%d, %d, %d, %d)' %(TP, TN, FP, FN))
    print('sensitivity: {:.1%}'.format(TP/float(TP+FN)))
    print('specificity: {:.1%}'.format(TN/float(TN+FP)))
    print('precision: {:.1%}'.format(TP/float(TP+FP)))
    print('F1 score: {:.1%}'.format(2*TP/float(TP+FP)*TP/float(TP+FN) / (TP/float(TP+FP)+TP/float(TP+FN))))
    return prob_list

In [14]:
# build the tensorflow graph
image_size = 40
batch_size = 512
patch_size = 5
depth = 32
depth2 = 64
depth3 = 64
num_hidden = 512
num_hidden2 = 256
keep_prob = 0.7

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.constant(0.1, shape=[depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth2], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(0.1, shape=[depth2]))
    layer3_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth2, depth3], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(0.1, shape=[depth3]))   
    layer4_weights = tf.Variable(tf.truncated_normal(
      [image_size // 8 * image_size // 8 * depth3, num_hidden], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(0.1, shape=[num_hidden]))
    layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden2], stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(0.1, shape=[num_hidden2]))
    layer6_weights = tf.Variable(tf.truncated_normal(
      [num_hidden2, num_labels], stddev=0.1))
    layer6_biases = tf.Variable(tf.constant(0.1, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        bias1 = tf.nn.relu(conv1 + layer1_biases)
        pool1 = tf.nn.max_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
        bias2 = tf.nn.relu(conv2 + layer2_biases)
        pool2 = tf.nn.max_pool(bias2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        conv3 = tf.nn.conv2d(pool2, layer3_weights, [1, 1, 1, 1], padding='SAME')
        bias3 = tf.nn.relu(conv3 + layer3_biases)
        pool3 = tf.nn.max_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        shape = pool3.get_shape().as_list()
        reshape = tf.reshape(pool3, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer4_weights) + layer4_biases)
        hidden2 = tf.nn.relu(tf.matmul(hidden, layer5_weights) + layer5_biases)
        drop3 = tf.nn.dropout(hidden2, keep_prob)
        return tf.matmul(drop3, layer6_weights) + layer6_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.01, global_step, 1000, 0.85, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [15]:
# first iteratively train the model
# then calculate the prediction accuracy on the test set using the trained model
# and generate the confusion matrix
num_steps = 20001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    prob = confusion(test_prediction.eval(), test_labels)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 2.455675
Minibatch accuracy: 46.9%
Validation accuracy: 50.0%
Minibatch loss at step 1000: 0.485365
Minibatch accuracy: 75.6%
Validation accuracy: 76.5%
Minibatch loss at step 2000: 0.309335
Minibatch accuracy: 88.7%
Validation accuracy: 86.2%
Minibatch loss at step 3000: 0.167716
Minibatch accuracy: 94.5%
Validation accuracy: 91.3%
Minibatch loss at step 4000: 0.147194
Minibatch accuracy: 95.3%
Validation accuracy: 93.3%
Minibatch loss at step 5000: 0.065614
Minibatch accuracy: 98.4%
Validation accuracy: 95.5%
Minibatch loss at step 6000: 0.059642
Minibatch accuracy: 98.0%
Validation accuracy: 96.5%
Minibatch loss at step 7000: 0.044985
Minibatch accuracy: 97.9%
Validation accuracy: 97.3%
Minibatch loss at step 8000: 0.036545
Minibatch accuracy: 99.0%
Validation accuracy: 98.1%
Minibatch loss at step 9000: 0.031455
Minibatch accuracy: 99.4%
Validation accuracy: 98.4%
Minibat

In [16]:
# print the first 10 predictions with the corresponding probabilities.
# each tuple has the format: (true label, model prediction, prediction probability)
print(prob[:10])

[(0, 1, 0.59565789), (0, 0, 0.9723103), (0, 0, 1.0), (1, 1, 0.98488754), (1, 1, 0.99974436), (0, 0, 0.99924242), (1, 1, 0.93535197), (1, 1, 0.64920932), (0, 0, 0.99983823), (0, 0, 0.99837619)]
